# Imports

In [1]:
import keras
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy.io import wavfile

from sklearn.metrics import accuracy_score, hamming_loss, zero_one_loss, auc

from bokeh.plotting import figure, show
from bokeh.io import output_notebook

from sklearn.metrics import roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt
import random

import librosa
import pickle as pk

from keras.layers import Conv2D, MaxPool2D, Activation, Dense, Input, Flatten, BatchNormalization, Dropout
from keras.losses import binary_crossentropy
from keras.optimizers import SGD
from keras.utils import Sequence
import keras.backend as K
import tensorflow as tf

Using TensorFlow backend.


###### Parameters

In [10]:
# Paths
dataset_dir = '../data/MagnaTagATune/MEL_default_hop/'
annotations_path = '../data/MagnaTagATune/annotation_reduced_50.csv'

checkpoint_dir = './checkpoints_RAW/'
checkpoint_file_name = 'weights-{epoch:03d}-{val_loss:.5f}.hdf5'
log_dir ='./logs_MEL_32f/'

# Functions

###### Align dataset split to batch size

In [11]:
def align_split(split, batch_size, num_songs):
    num_songs_split = split*num_songs
    return int(num_songs_split - num_songs_split%batch_size)/num_songs

###### Data reading during training

In [12]:
class MagnaTagATuneSequence(Sequence):

    def __init__(self, train_set_paths, train_set_labels, batch_size):
        self.paths, self.y = train_set_paths, train_set_labels
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.paths) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x_paths = self.paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_x = []
        for value in batch_x_paths:
            path = dataset_dir + value[:-3]+'p'
            S = pk.load(open(path,'rb'))
            batch_x.append(S)
        batch_x = np.array(batch_x)[:,:,:,np.newaxis]        
        return (batch_x,batch_y)    

###### Performance Metrics (not used anymore)

In [13]:
def ratio_wrong_over_correct_ones(y_true, y_pred):
    op1 = K.sum(K.abs(K.cast(y_true - K.round(y_pred), dtype='float32')))
    op2 = K.sum(K.cast(K.equal(y_true,1.0),dtype='float32'))
    return op1/op2

def ratio_correct_ones(y_true, y_pred):
    op1 = K.sum(K.cast(K.equal(y_true + K.round(y_pred+0.15),2.0),dtype='float32'))
    op2 = K.sum(K.cast(K.equal(y_true,1.0),dtype='float32'))
    return op1/op2

def auc_roc(y_true, y_pred):
    # any tensorflow metric
    value, update_op = tf.metrics.auc(y_true, y_pred, summation_method='careful_interpolation')

    # find all variables created for this metric
    metric_vars = [i for i in tf.local_variables() if 'auc_roc' in i.name.split('/')[1]]

    # Add metric variables to GLOBAL_VARIABLES collection.
    # They will be initialized for new session.
    for v in metric_vars:
        tf.add_to_collection(tf.GraphKeys.GLOBAL_VARIABLES, v)

    # force to update metric values
    with tf.control_dependencies([update_op]):
        value = tf.identity(value)
        return value


###### Best checkpoint selection

In [6]:
def find_best_checkpoint(prev_chkpts):
    best_ratio = np.inf
    best_chkpt = ''
    best_epoch = 0
    for chkpt in prev_chkpts:
        epoch = int(chkpt[8:11])
        ratio = float(chkpt[12:19])
        
        if ratio < best_ratio:
            best_ratio = ratio
            best_chkpt = chkpt
            best_epoch = epoch
    print('\n starting from model {} \n'.format(best_chkpt))
    return best_chkpt, best_epoch

# Preparation

###### Prepare Dataset

In [7]:
annotations = pd.read_csv(annotations_path, sep='\t')

tot_t_size = 0.866203
tot_train_set, test_set = train_test_split(annotations, train_size=tot_t_size, test_size=(1-tot_t_size), random_state=42) 

print("Complete Train set size: {}".format(tot_train_set.shape[0]))
print("Test set size: {} \n".format(test_set.shape[0]))

t_size = 0.91429
train_set, val_set = train_test_split(tot_train_set, train_size=t_size, test_size=(1-t_size), random_state=42) 

print("Train set size: {}".format(train_set.shape[0]))
print("Validation set size: {} \n".format(val_set.shape[0]))

train_set_paths = train_set['mp3_path'].values
train_set_labels = train_set.drop(columns=['mp3_path','Unnamed: 0']).values

y_dimension = train_set_labels.shape[1]
S = pk.load(open(dataset_dir + annotations['mp3_path'][0][:-3]+ 'p','rb'))
x_dimension = S.shape

print("X dimension: {}\nY dimension: {} \n".format(x_dimension, y_dimension))

'''   
val_set_paths = val_set['mp3_path'].values
val_set_labels = val_set.drop(columns=['mp3_path','Unnamed: 0']).values
    

    
print('\n* * * Loading Validation Set into Memory * * *\n')

val_set_data = []
for value in tqdm(val_set_paths):
    path = dataset_dir+value[:-3]+'p'
    S = pk.load(open(path,'rb'))
    val_set_data.append(S)  
val_set_data = np.array(val_set_data)[:,:,:,np.newaxis] 
'''

Complete Train set size: 22400
Test set size: 3460 

Train set size: 20480
Validation set size: 1920 

X dimension: (128, 911)
Y dimension: 50 



"   \nval_set_paths = val_set['mp3_path'].values\nval_set_labels = val_set.drop(columns=['mp3_path','Unnamed: 0']).values\n    \n\n    \nprint('\n* * * Loading Validation Set into Memory * * *\n')\n\nval_set_data = []\nfor value in tqdm(val_set_paths):\n    path = dataset_dir+value[:-3]+'p'\n    S = pk.load(open(path,'rb'))\n    val_set_data.append(S)  \nval_set_data = np.array(val_set_data)[:,:,:,np.newaxis] \n"

###### Prepare Test Set

In [8]:
test_set_paths = test_set['mp3_path'].values
test_set_labels = test_set.drop(columns=['mp3_path','Unnamed: 0']).values
test_set_size = len(test_set_paths)
print("Test set size: {} ".format(test_set_size))

Test set size: 3460 


###### Load best Model

In [9]:
n_gpus = 1
batch_size = 32

previous_checkpoints = os.listdir(checkpoint_dir)
best_checkpoint, best_epoch = find_best_checkpoint(previous_checkpoints)
#model.load_weights(checkpoint_dir + best_checkpoint)
model = keras.models.load_model(checkpoint_dir + best_checkpoint)
parallel_model = keras.utils.multi_gpu_model(model, gpus=n_gpus)

FileNotFoundError: [Errno 2] No such file or directory: './checkpoints_MEL_32f/'

###### Prediction and evaluation

In [10]:
predictions = parallel_model.predict_generator(MagnaTagATuneSequence(test_set_paths, test_set_labels, batch_size), verbose=1)
#predictions = parallel_model.predict(test_set_data,batch_size=batch_size,verbose=1)

109/109 [==============================] - 491s 5s/step


In [11]:
try:
    roc_auc = roc_auc_score(test_set_labels, predictions)
    print("Test roc auc result: {} ".format(roc_auc))
except Error as e:
    print(e)

Test roc auc result: 0.8932105996027028 


In [30]:
fpr = dict()
tpr = dict()
thresholds = dict()
ratio = dict()
roc_auc = dict()
best_th = dict()
beast_auc = dict()
for i in range(test_set_labels.shape[1]):
    fpr[i], tpr[i], thresholds[i] = roc_curve(test_set_labels[:, i], predictions[:, i])
    ratio[i] = tpr[i]/fpr[i]
    best_th[i] = thresholds[i][np.argmax(ratio[i])]
    best_auc[i] = auc(fpr[i][np.argmax(ratio[i])],tpr[i][np.argmax(ratio[i])])
    roc_auc[i] = auc(fpr[i], tpr[i])
    
    
fpr["micro"], tpr["micro"], _ = roc_curve(test_set_labels.ravel(), predictions.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])



plt.figure()
lw = 3
label = 2
plt.plot(fpr[label], tpr[label], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[label])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()
print(best_th)

/home/emanuele.dallalonga/miniconda3/envs/musictagger/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


TypeError: Singleton array 0.0 cannot be considered a valid collection.

In [ ]:
test = predictions[0]
print("All the same: {}".format(all([all(i) for i in [test-i<np.finfo(np.float32).eps for i in predictions]])))

In [ ]:
a=501
print(predictions[a])
print(test_set_labels[a])

d = dict()
for idx in range(y_dimension):
    
K.eval(ratio_correct_ones(y_true=K.variable(test_set_labels), y_pred=K.variable(predictions)))

In [ ]:
import math
weights = np.concatenate([ i.flatten() for i in model.get_weights() ])
print('Are there NaN weights? {}'.format(any([ math.isnan(i) for i in weights])))